In [38]:
import pandas as pd
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data
import numpy as np

In [34]:
#metadata and count table
metadata = pd.read_table('../../adonis/metadata_12dataset_1280_adonis_number_gender.txt', sep = "\t")
metadata_CD = metadata.loc[(metadata['Study'] == 'Franzosa2019IBD_CD')]
metadata_CD = metadata_CD.set_index ('featureid')
metadata_CD = metadata_CD.sort_index()
table = pd.read_table('../../Meta_diseases_analyses/txt/CD_AD.txt', sep = "\t", skiprows=1)
table = table.set_index ('#OTU ID')
table = table[list(metadata_CD.index)]

In [35]:
table_20genes = table.head(20)
table_20genes = table_20genes.T
table_20genes = table_20genes.sort_index()
table_20genes

#OTU ID,ko:K00001,ko:K00002,ko:K00003,ko:K00004,ko:K00005,ko:K00006,ko:K00007,ko:K00008,ko:K00009,ko:K00010,ko:K00011,ko:K00012,ko:K00013,ko:K00014,ko:K00015,ko:K00016,ko:K00018,ko:K00019,ko:K00020,ko:K00021
SRR6468502,3788521.0,4252331.0,21510389.0,2037312.0,18760322.0,0.0,26742.0,20295263.0,9040350.0,6437164.0,13.0,14877893.0,17005192.0,28130022.0,2114224.0,22530641.0,18928176.0,682534.0,7037251.0,9882.0
SRR6468505,4281696.0,558050.0,8775543.0,186098.0,2619603.0,0.0,931.0,5758005.0,8129387.0,598568.0,88.0,21596969.0,13218242.0,16168936.0,1215613.0,6393645.0,11227109.0,1622518.0,1680971.0,98380.0
SRR6468507,5375549.0,323271.0,20194996.0,543147.0,5555903.0,0.0,12950.0,8120606.0,3429551.0,1570910.0,0.0,13416725.0,15351136.0,23235285.0,32364.0,20155224.0,5681086.0,913856.0,1281261.0,4725.0
SRR6468514,5802977.0,617821.0,18315103.0,373672.0,8324607.0,0.0,2192.0,12265860.0,2186728.0,1015095.0,0.0,14638854.0,14449176.0,20238558.0,356971.0,13930470.0,9266335.0,525120.0,1790593.0,21636.0
SRR6468517,5062154.0,779621.0,19144228.0,678775.0,10436428.0,0.0,1793.0,15015987.0,4692911.0,967000.0,0.0,6570229.0,10886107.0,23445707.0,110474.0,16796051.0,6119427.0,491383.0,3758713.0,11472.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR6468707,2659998.0,864282.0,16735436.0,109128.0,7663078.0,0.0,2545.0,9116731.0,1388804.0,2785592.0,0.0,10240187.0,12587178.0,17559760.0,59833.0,19627304.0,7151227.0,439992.0,4661382.0,18707.0
SRR6468709,3577829.0,409632.0,13185827.0,150511.0,4180944.0,0.0,835.0,7956486.0,5633587.0,1249355.0,69.0,16417921.0,12462325.0,17874010.0,1929900.0,9627176.0,11148642.0,1283993.0,1509603.0,152474.0
SRR6468711,2185426.0,168015.0,4972053.0,198549.0,2859093.0,0.0,11365.0,3714761.0,3525877.0,628974.0,0.0,7121456.0,5462014.0,8161419.0,92802.0,4562433.0,4619714.0,976973.0,811595.0,58196.0
SRR6468712,5059494.0,793229.0,7240698.0,946442.0,3097564.0,0.0,2506.0,5810215.0,1107060.0,1509983.0,31.0,16587180.0,8387735.0,10607612.0,789834.0,9703648.0,7451386.0,378104.0,1602384.0,65257.0


In [36]:
import pydeseq2

def pydeseq2_permutation_test(counts, group_labels, num_permutations=10):
    """Permutation test for differential expression using PyDESeq2.

    Parameters
    ----------
    counts : array_like
        The count matrix of shape (n_samples, n_genes).
    group_labels : array_like
        The group labels of shape (n_samples,).
    num_permutations : int, optional (default=1000)
        The number of permutations to perform.

    Returns
    -------
    p_values : array_like
        The p-values of the permutation test for each gene.
    """
    # Convert counts and group labels to PyDESeq2 input format
    dds = DeseqDataSet(
    counts,
    group_labels,
    design_factors="disease",  # compare samples based on the "disease"
    # column ("B" vs "A")
    refit_cooks=True,
    n_cpus=8,
)
    dds.deseq2()
    stat_res = DeseqStats(dds, n_cpus=8)
    # Get the test statistics for each gene
    res = stat_res.summary()
    
    # Initialize the null distribution of the test statistic
    null_distribution = np.zeros((num_permutations, counts.shape[1]))

    # Perform the permutations
    for i in range(num_permutations):
        # Shuffle the group labels
#         permuted_labels = np.random.permutation(group_labels)
#         permuted_labels = pd.DataFrame(permuted_labels)
        permuted_labels = np.random.permutation(group_labels['disease'])
        permuted_labels = pd.DataFrame(permuted_labels, index=group_labels.index)
        permuted_labels = pd.DataFrame(permuted_labels)
        permuted_labels.columns = ['disease']
        # Convert the permuted group labels to PyDESeq2 input format
        # Run the DESeq2 analysis with the permuted group labels
        counts.reindex(permuted_labels.index)
        permuted_dds = DeseqDataSet(
        counts, 
        permuted_labels, 
        design_factors="disease")
        permuted_dds.deseq2()
        permuted_stat_res = DeseqStats(permuted_dds, n_cpus=8)
        # Get the test statistics for each gene with the permuted group labels
        permuted_res = permuted_stat_res.summary()

        # Add the permuted test statistic to the null distribution
        null_distribution[i, :] = permuted_stat_res.results_df['log2FoldChange']

    # Calculate the p-values for each gene
    p_values = np.zeros(counts.shape[1])
    for i in range(counts.shape[1]):
        if stat_res.results_df['log2FoldChange'].values[i] > 0:
            p_values[i] = np.mean(null_distribution[:, i] > stat_res.results_df['log2FoldChange'].values[i])
        else:
            p_values[i] = np.mean(null_distribution[:, i] < stat_res.results_df['log2FoldChange'].values[i])

    return p_values

In [39]:
pydeseq2_permutation_test(table_20genes, metadata_CD, num_permutations=10)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.02 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 2 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.01 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.989640e+06,-0.414489,0.132655,-3.124574,1.780625e-03,0.003957
ko:K00002,9.003372e+05,-0.168351,0.176373,-0.954514,3.398238e-01,0.399793
ko:K00003,1.725142e+07,0.249343,0.101834,2.448514,1.434468e-02,0.028689
ko:K00004,8.929586e+05,-0.688170,0.179465,-3.834557,1.257909e-04,0.000629
ko:K00005,7.161609e+06,-0.292383,0.091469,-3.196507,1.391023e-03,0.003478
ko:K00006,2.816531e+01,0.547728,0.768327,0.712884,4.759173e-01,0.514934
ko:K00007,1.713274e+04,-1.823705,0.359277,-5.076039,3.853841e-07,0.000004
ko:K00008,1.194929e+07,0.050600,0.085166,0.594135,5.524219e-01,0.552422
ko:K00009,6.538020e+06,0.156448,0.152223,1.027759,3.040632e-01,0.380079


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.60 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.501453e+06,0.059275,0.107912,0.549287,0.582808,0.832584
ko:K00002,9.003372e+05,0.127828,0.176750,0.723213,0.469549,0.783093
ko:K00003,1.725142e+07,-0.098735,0.104168,-0.947839,0.343211,0.762692
ko:K00004,8.929586e+05,0.370655,0.186972,1.982415,0.047433,0.247968
ko:K00005,7.161609e+06,0.064742,0.095457,0.678229,0.497627,0.783093
ko:K00006,2.816531e+01,1.072599,0.762459,1.406763,0.159498,0.455708
ko:K00007,1.971395e+04,0.316787,0.399632,0.792697,0.427954,0.783093
ko:K00008,1.194929e+07,-0.035104,0.085905,-0.408635,0.682807,0.879465
ko:K00009,6.538020e+06,-0.100838,0.152697,-0.660379,0.509011,0.783093


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.19 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


...

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.37 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.13 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.501453e+06,-0.280618,0.104777,-2.678238,0.007401,0.049340
ko:K00002,9.003372e+05,0.055458,0.177067,0.313204,0.754126,0.859660
ko:K00003,1.725142e+07,0.040251,0.103814,0.387720,0.698223,0.859660
ko:K00004,8.929586e+05,-0.093768,0.189739,-0.494196,0.621167,0.859660
ko:K00005,7.161609e+06,0.172387,0.094282,1.828424,0.067486,0.337430
ko:K00006,5.209458e+01,-3.072788,0.770103,-3.990101,0.000066,0.001321
ko:K00007,1.971395e+04,0.422172,0.398911,1.058312,0.289913,0.859660
ko:K00008,1.194929e+07,0.015833,0.086004,0.184099,0.853936,0.859660
ko:K00009,6.538020e+06,-0.142760,0.152431,-0.936560,0.348985,0.859660


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.48 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.501453e+06,-0.215928,0.106099,-2.035158,0.041835,0.209175
ko:K00002,9.003372e+05,0.132826,0.176669,0.751837,0.452149,0.753582
ko:K00003,1.725142e+07,0.017941,0.104579,0.171551,0.863790,0.909253
ko:K00004,8.929586e+05,-0.251826,0.188507,-1.335902,0.181581,0.412536
ko:K00005,7.161609e+06,-0.125630,0.094917,-1.323585,0.185641,0.412536
ko:K00006,4.208974e+01,-1.323511,0.794122,-1.666635,0.095587,0.273106
ko:K00007,1.407568e+04,-0.948269,0.359178,-2.640107,0.008288,0.082880
ko:K00008,1.194929e+07,0.054838,0.085824,0.638964,0.522847,0.804379
ko:K00009,6.538020e+06,-0.258772,0.151076,-1.712864,0.086738,0.273106


array([0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0.3, 0.2, 0.3, 0. , 0.1, 0. ,
       0.1, 0. , 0.1, 0.1, 0. , 0. , 0. ])